In [1]:
"""
univ3-liquidity.ipynb

Goal with this notebook is to plot the liquidity distribution
of a given pool over time.

Should be able to view a 2-D snapshot of liquidity at a particular
block (with time slider) and, more generally, a 3D plot of the full
liquidity distribution history over time.
"""
# switch to univ3-ape wd first for ape project to work properly
import os
from pathlib import Path

if Path(*Path(os.getcwd()).parts[-3:]) == Path('univ3-ape/notebook/queries'):
    os.chdir('../..')

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import typing as tp
from ape import accounts, chain, Contract, networks, project

In [4]:
# SEE: https://gist.github.com/banteg/dcf6082ff7fc6ad51ce220146f29d9ff
networks.parse_network_choice('ethereum:mainnet:alchemy').__enter__()

<alchemy chain_id=1>

In [5]:
univ3 = project.dependencies['UniswapV3Core']['main']

In [6]:
# SEE: https://docs.uniswap.org/contracts/v3/reference/deployments
factory = Contract("0x1F98431c8aD98523631AE4a59f267346ea31F984")
factory

<UniswapV3Factory 0x1F98431c8aD98523631AE4a59f267346ea31F984>

In [7]:
head_block = chain.blocks.head.number

In [8]:
deploy_block = 12369621  # block when factory deployed
step = 7200  # sample liquidity distribution once per day
start_block = head_block - 7200 * 7 # last 7 days ... TODO: last 6 months

In [9]:
# relevant block numbers to query for
blocks = range(start_block, head_block, step)
blocks

range(16577835, 16628235, 7200)

In [10]:
pool = univ3.UniswapV3Pool.at("0xa3f558aebAecAf0e11cA4b2199cC5Ed341edfd74")  # LDO/ETH 0.3%

In [11]:
print(f"token0: {pool.token0()}, token1: {pool.token1()}, fee: {pool.fee()}")

token0: 0x5A98FcBEA516Cf06857215779Fd812CA3beF1B32, token1: 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2, fee: 3000


In [12]:
slot0 = pool.slot0()
slot0

slot0_return(sqrtPriceX96=3238180615412954133714266404, tick=-63950, observationIndex=0, observationCardinality=1, observationCardinalityNext=1, feeProtocol=0, unlocked=True)

In [13]:
liquidity = pool.liquidity()
tick_spacing = pool.tickSpacing()

In [14]:
TICK_MATH_MAX_TICK = 887272
max_tick = TICK_MATH_MAX_TICK - (TICK_MATH_MAX_TICK % tick_spacing)
min_tick = -max_tick
print(f"max_tick: {max_tick}, min_tick: {min_tick}")

max_tick: 887220, min_tick: -887220


In [18]:
# relevant ticks to check liquidity for
ticks = range(min_tick, max_tick, tick_spacing)
ticks

range(-887220, 887220, 60)

In [ ]:
%time

# query for tick data each block and insert into pandas dataframe
FILENAME = f"notebook/data/liquidity_{pool.address}_{start_block}_{head_block}_{step}.csv"
is_head = True
for block in blocks:
    for tick in ticks:
        print(f"Processing tick {tick} at block {block} ...", end='\r')
        row = {'block_number': [block], 'tick': [tick]}
        
        # get the liquidity data for tick at block
        ret = pool.ticks(tick, block_identifier=block)
        data = {k: [v] for k, v in ret.items()}
        row.update(data)
        
        # convert to pd dataframe then append to file
        df = pd.DataFrame(data=row)
        df.to_csv(FILENAME, mode='a', index=False, header=is_head)
        
        if is_head:
            is_head = False